In [1]:
from processing.utils import *
from processing.utils2 import *
from processing.utils3 import *
from processing.gpcrdb_soup import *
from processing.download import *
from processing.processor import *
from processing.df_to_cif import *
from processing.bondtypes import *
from processing.affinities import *
from analysis.analysis import *


from torch_geometric.data import Data
import numpy as np
import torch
from scipy.spatial.distance import cdist, pdist
from tqdm import trange

In [2]:
# Filtering etc is done with the processor!

In [3]:
p = CifProcessor()

In [4]:
p.read_pkl_metainfo()

In [5]:
p.read_pkl(mode='rg', folder='data/processed/')

  0%|                                                                                                                                                                                                             | 0/545 [00:00<?, ?it/s]

Reading files with generic numbers on receptors and gproteins.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 545/545 [02:44<00:00,  3.31it/s]


In [6]:
BOND_TYPE_DF

,H,N,C,O,S
H,0,1,2,3,4
N,1,5,6,7,8
C,2,6,9,10,11
O,3,7,10,12,13
S,4,8,11,13,14


In [7]:
FULL_ATOM_LIST = list(p.dfl[0]['label_atom_id'].unique())

In [14]:
class GraphProcessor:
    def __init__(self,
                 d: list,
                 p: CifProcessor,
                 label_type = 'Log(Emax/EC50)'):
        self.d = d
        self.p = p
        self.allow_exception=False
        
        self.simplified = False
        if 'generic_position' in list(p.dfl[0].columns):
            self.simplified=True
        
        self.atom_list = FULL_ATOM_LIST
        self.edge_criteria = ['self', 'distance']
        self.edge_features = ['unitary', 'distance', 'EM']
        self.node_criteria = ['H5']
    
    # ================================================================================================================
    
    def simplify(self):
        if not self.simplified:
            for i in trange(len(self.p.dfl)):
                if self.allow_exception:
                    try:
                        df = self.p.dfl[i]
                        self.p.dfl[i] = pd.DataFrame()
                        self.p.dfl[i] = self._simplify_gen_pos(df).reset_index()
                    except:
                        pass
                else:
                    df = self.p.dfl[i]
                    self.p.dfl[i] = pd.DataFrame()
                    self.p.dfl[i] = self._simplify_gen_pos(df).reset_index()
                self.simplified = True
        else:
            pass
        
    def _simplify_gen_pos(self, df):
        def sgp(gen_pos='', gprot_pos=''):
            if gprot_pos != '':
                return gprot_pos
            else:
                return gen_pos.split('x')[0]
        cols = list(df.columns)
        if ('gen_pos' in cols) and ('gprot_pos' in cols):
            df['generic_position'] = df.apply(lambda x: sgp(x.gen_pos, x.gprot_pos), axis=1)
            df.drop(['group_PDB', 'label_seq_id', 'label_asym_id',
                    'auth_seq_id', 'id',
                    'phi', 'omega', 'psi', 'label_comp_id',
                    'label_2_uni', 'gen_pos', 'gen_pos1', 'gen_pos2', 'gprot_pos',
                    'uniprot_comp_id', 'fam_comp_id', 'uniprot_comp_sid'], axis=1, inplace=True)
            return df
        elif 'gen_pos' in cols:
            df['generic_position'] = df.apply(lambda x: sgp(x.gen_pos), axis=1)
            df.drop(['group_PDB', 'label_seq_id','label_asym_id',
                   'auth_seq_id', 'id',
                   'phi', 'omega', 'psi','label_comp_id',
                   'label_2_uni', 'gen_pos', 'gen_pos1', 'gen_pos2',
                   'uniprot_comp_sid'], axis=1, inplace=True)
            return df
        else:
            df['generic_position'] = ''
            df.drop(['group_PDB', 'label_seq_id','label_asym_id',
                   'auth_seq_id', 'id','label_comp_id',
                   'phi', 'omega', 'psi'], axis=1, inplace=True)   
            return df
        
    # ================================================================================================================
    
    def assign_labels(self, label_type = 'Log(Emax/EC50)'):
        A = AffinityProcessor(setting='families')
        A.set_label_type(label_type)
        A.set_group()
        prev_len = len(self.p.table)
        print("Filtering out samples with no associated affinity value...")
        self.p = filter_valid_pdbs_with_affinities(self.p, A)
        print("Retaining {} / {} samples!".format(len(self.p.table), prev_len))
        label_df = make_label_df(self.p, A, label_type)
        self.p.table = pd.merge(self.p.table, label_df, on='PDB')
    
    # ================================================================================================================
    
    def set_atom_list(self, filtered_atom_list=['CA']):
        self.atom_list = filtered_atom_list
        print("Set atom_list to {}.".format(self.atom_list))
    
    def apply_atom_list_filter(self):
        for i in range(len(self.p.dfl)):
            p.dfl[i] = p.dfl[i][p.dfl[i]['label_atom_id'].isin(self.atom_list)]
        print("Filtered p by atom_list.")
        
    # ================================================================================================================
    
    def create_graph(self, 
                     node_criteria='H5', 
                     edge_criteria='radius',
                     h5start=13,
                     h5radius=7,
                     max_edge_dist=7):
        self.selection = []
        self.edges = []
        self.edge_attrs = []
        for i in trange(len(self.p.dfl)):
            # INITIALIZATION
            df = self.p.dfl[i]
            self._select_edge_criterion()
            self._select_edge_features()
            self._select_node_criterion()
            # NODE SELECTION
            df_idxs = self._get_selection(df,
                                          h5start=h5start,
                                          h5radius=h5radius,)
            df_sele = self._filter_by_index_list(df, df_idxs)
            if len(df_sele) > 0:
                self.selection.append(df_sele)
                # EDGES: LIST
                edges = self._create_edge_list(df=df_sele, max_edge_dist=max_edge_dist)
                self.edges.append(edges)
                # EDGES: ATTRIBUTES
                edge_attr = self._get_edge_features(df_sele, edges, max_edge_dist)
                self.edge_attrs.append(edge_attr)
                y = self._get_label(df_sele)
                # Todo: get the node features (Z or atom-label (or both))
                x = df_sele.apply(lambda x: RES_DICT[x.label_comp_sid], axis=1).to_numpy().astype(np.int8)
                pos = df_sele[['Cartn_x', 'Cartn_y', 'Cartn_z']].to_numpy().astype(float)
                gp.d.append(self._get_data_object(x, edges, edge_attr, y, pos))
                print('\n\n\n')
            
            
    # ----------------------------------------------------------------------------------------------------------------
    
    def _select_edge_criterion(self, edge_criteria=[], edge_crit_dist=5):
        if not isinstance(edge_criteria, list):
            edge_criteria = [edge_criteria]
        if len(edge_criteria) > 0:
            self.edge_criteria = edge_criteria
        print("Selected edge criterion:", self.edge_criteria)
    
    def _select_edge_features(self, edge_features=[]):
        if not isinstance(edge_features, list):
            edge_features = [edge_features]
        if len(edge_features) > 0:
            self.edge_features = edge_features
        print("Selected edge features:", self.edge_features)
        
    def _select_node_criterion(self, node_criteria=[]):
        if not isinstance(node_criteria, list):
            node_criteria = [node_criteria]
        if len(node_criteria) > 0:
            self.node_criteria = node_criteria
        print("Selected criterion for node selection:", self.node_criteria)
              
    # ----------------------------------------------------------------------------------------------------------------
    
    def _get_selection(self, 
                       df: pd.DataFrame,
                       filter_by_chain=True,
                       gpcr=True,
                       gprotein=True,
                       h5start=13,
                       h5radius=7):
        # get a list of nodes (you can have multiple criterions met at the same time...)
        df.reset_index(inplace=True, drop=True)
        valid = []  # indices filtered by chain
        idxs = []  # list of selected nodes
        df_idxs = []  # list of selected nodes filtered by chain
        
        if filter_by_chain:
            print("Filtering by chain ---- TBD not implemented!")
            if gpcr:
                valid += self._get_gpcr(df)
            if gprotein:
                valid += self._get_gprotein(df)
        else:
            valid = list(df.index)
        
        if len(valid) >= 0:
            if 'H5' in self.node_criteria:
                print("Searching nodes meeting H5 criterion...")
                idxs += self._get_h5(df, h5start, h5radius)
            if 'Interaction Surface' in self.node_criteria:
                print("Searching nodes in the interaction surface...")
                idxs += self._get_interaction_surface(df)
            if 'Interaction Site' in self.node_criteria:
                print("Searching nodes in the interaction site...")
                idxs += self._get_interaction_site(df)
            for idx in idxs:
                if idx in valid:
                    df_idxs.append(idx)
            return idxs
        else:
            print("No valid nodes found!")
            return []
    
    def _get_gpcr(self, df):
        # any chain with labels that mark it as a gpcr
        labelled_chain = ['A']
        return []
    
    def _get_gprotein(self, df):
        # any chain with labels that mark it as a gprotein ~ i.e. generic residue number of said chain contains 'G.'
        labelled_chain = ['A']
        return []
    
    def _get_h5(self, df, h5start, h5radius):
        # any chain with 
        print("Checking for nodes within H5 radius:", h5radius)
        strs = ['G.H5.' + str(x) for x in range(27) if ((x >= h5start) & (x <= 26))]
        df_h5 = df[df['generic_position'].isin(strs)]
        xyz_full = df[['Cartn_x', 'Cartn_y', 'Cartn_z']].to_numpy().astype(float)
        xyz_h5 = df_h5[['Cartn_x', 'Cartn_y', 'Cartn_z']].to_numpy().astype(float)
        if len(xyz_h5) > 0:
            D = cdist(xyz_full, xyz_h5).T
            h5_list = []
            for p in range(D.shape[1]):
                if np.any(D[:, p] < h5radius):
                    h5_list.append(p)
            return h5_list
        else:
            return []
    
    def _get_interaction_surface(self, df, max_dist=7, n_neighbor_res=2):
        pass
    
    def _get_interaction_site(self, df, cons_r_res=['2.50', '3.50', '7.53'], radius=20):
        pass
    
    def _filter_by_index_list(self, df, df_idxs):
        if len(df_idxs) > 0:
            filtered = df.loc[df.index[df_idxs]].reset_index(drop=True)
            return filtered
        else:
            return pd.DataFrame()
    
    # ----------------------------------------------------------------------------------------------------------------
    
    def _create_edge_list(self, 
                          df: pd.DataFrame,
                          max_edge_dist=7,
                          ):
        """
        Return a list of shape (2, number_of_edges)
        """
        edge_list = []
        if 'residual' in self.edge_criteria:
            # add connection between all atoms within the same residue
            pass
        if 'EM' in self.edge_criteria:
            pass
        if 'distance' in self.edge_criteria:
            xyz = df[['Cartn_x', 'Cartn_y', 'Cartn_z']].to_numpy().astype(float)
            D = cdist(xyz, xyz, 'euclidean')
            edge_dist_list = []
            for pi in range(D.shape[0]):
                if 'self' not in self.edge_criteria:
                    for pj in range(D.shape[1]):
                        D[i, i] = max_edge_dist + 1  # this removes all self interactions
                edges = np.where(D[:, pi] <= max_edge_dist)[0].tolist()
                for e in edges:
                    edge_dist_list.append((pi, e))
            edge_list += edge_dist_list
        elif 'self' in self.edge_criteria:
            # each node interacts with itself
            idxs = list(df.index)
            self_interactions = [(x, x) for x in idxs]
            edge_list += self_interactions
        return edge_list
        
        
    def _get_em_dist(self,
                     xyz1,
                     z1,
                     xyz2,
                     z2
                    ):
        em_dists = []
        # z1 * z2 / (dist^6)
        return em_dists
        
    # ----------------------------------------------------------------------------------------------------------------
        
    def _get_edge_features(self, df, edge_list, max_edge_dist):
        # unitary? --> 1 (for every edge)
        edge_features = []
        for _, (i, j) in enumerate(edge_list):
            features = []
            xyzi = df.iloc[i][['Cartn_x', 'Cartn_y', 'Cartn_z']].to_numpy().astype(float)
            xyzj = df.iloc[j][['Cartn_x', 'Cartn_y', 'Cartn_z']].to_numpy().astype(float)
            if 'unitary' in self.edge_features:
                features += [1]
            # distance? --> (euclidean distance)
            if 'distance' in self.edge_features:
                dist = np.linalg.norm(xyzi-xyzj)
                features += [dist / max_edge_dist]
            # EM?  --> z1 * z2 / r^6
            if 'em' in self.edge_features:
                # xyzi = df.iloc[i][]
                # xyzj = df.iloc[j][]
                features +=  [0]
            # atom-atom-interaction as a class (that we can embed)?
            if 'bond_embedding' in self.edge_features:
                features += [0]
            edge_features.append(features)
        return edge_features
              
    # ----------------------------------------------------------------------------------------------------------------

    def _get_label(self, df):
        return 0
              
    # ----------------------------------------------------------------------------------------------------------------
    
    def _get_data_object(self, x, edge_index, edge_attr, y, pos):
        """
        Input:
        data.x: Node feature matrix with shape [num_nodes, num_node_features]
        data.edge_index: Graph connectivity in COO format with shape [2, num_edges] and type torch.long
        data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
        data.y: Target to train against (may have arbitrary shape), e.g., node-level targets of shape [num_nodes, *] or graph-level targets of shape [1, *]
        data.pos: Node position matrix with shape [num_nodes, num_dimensions]
        """
        x = torch.tensor(x, dtype=torch.long)
        edge_index = torch.tensor(edge_index, dtype=torch.long)
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)
        y = torch.tensor(y, dtype=torch.long)
        pos = torch.tensor(pos, dtype=torch.float)
        return Data(x=x, edge_index=edge_index.T, edge_attr=edge_attr, y=y, pos=pos)
    
    
    # ================================================================================================================

In [15]:
gp = GraphProcessor(d=[], p=p)

In [16]:
gp.assign_labels()

Reading data from data/couplings/families_coupling.xls!
Initialized Affinity Processor!
Please set a group --------------  ['GPCRdb', 'Inoue', 'Bouvier'].
please set label type -----------  ['Guide to Pharmacology', 'Log(Emax/EC50)', 'pEC50', 'Emax'].

Selected label type 'Log(Emax/EC50)'.


Selected data of group 'GPCRdb'.

Filtering out samples with no associated affinity value...
Retaining 356 / 356 samples!


In [17]:
gp.set_atom_list()
gp.apply_atom_list_filter()

Set atom_list to ['CA'].
Filtered p by atom_list.


In [18]:
gp.simplify()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 356/356 [00:08<00:00, 41.95it/s]


In [19]:
eg = gp.p.dfl[0]

In [20]:
eg[eg['generic_position'] != '']

,index,PDB,auth_asym_id,label_atom_id,type_symbol,Cartn_x,Cartn_y,Cartn_z,identifier,label_comp_sid,generic_position
0,1,2R4R,A,CA,C,11.554,38.126,29.950,ADRB2_HUMAN,G,1.36
1,5,2R4R,A,CA,C,12.440,34.492,30.621,ADRB2_HUMAN,I,1.37
2,13,2R4R,A,CA,C,15.679,35.525,28.918,ADRB2_HUMAN,V,1.38
3,18,2R4R,A,CA,C,16.371,38.398,31.315,ADRB2_HUMAN,M,1.39
4,23,2R4R,A,CA,C,15.823,35.947,34.181,ADRB2_HUMAN,S,1.40
...,...,...,...,...,...,...,...,...,...,...,...
204,1514,2R4R,A,CA,C,37.484,35.712,41.431,ADRB2_HUMAN,Q,8.55
205,1523,2R4R,A,CA,C,40.023,33.349,42.973,ADRB2_HUMAN,E,8.56
206,1532,2R4R,A,CA,C,37.833,32.757,46.035,ADRB2_HUMAN,L,8.57
207,1540,2R4R,A,CA,C,35.902,30.369,43.791,ADRB2_HUMAN,L,8.58


In [30]:
gp.p.table[gp.p.table['PDB']=='2R4R'].drop(['Gs_y',
       'Gi/o_y', 'Gq/11_y', 'G12/13_y'], axis=1)[['Gs_x','Gi/o_x','Gq/11_x','G12/13_x']]

,Gs_x,Gi/o_x,Gq/11_x,G12/13_x
258,8.2,0.0,7.0,0.0


In [31]:
gp.create_graph(h5radius=14)

  4%|████████▊                                                                                                                                                                                          | 16/356 [00:00<00:02, 159.95it/s]

Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implement

 16%|███████████████████████████████▊                                                                                                                                                                   | 58/356 [00:00<00:01, 197.04it/s]

Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge fe

 22%|██████████████████████████████████████████▋                                                                                                                                                        | 78/356 [00:00<00:01, 198.02it/s]

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 352/356 [01:17<00:03,  1.04it/s]





Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14
Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 355/356 [01:19<00:00,  1.51it/s]





Selected edge criterion: ['self', 'distance']
Selected edge features: ['unitary', 'distance', 'EM']
Selected criterion for node selection: ['H5']
Filtering by chain ---- TBD not implemented!
Searching nodes meeting H5 criterion...
Checking for nodes within H5 radius: 14


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 356/356 [01:20<00:00,  4.44it/s]

In [ ]:
gp.selection[10]

In [ ]:
len(gp.edges[0])

In [ ]:
gp.d[0]

In [ ]:
import networkx as nx
from torch_geometric.utils.convert import to_networkx

In [ ]:
g = to_networkx(gp.d[10], to_undirected=True)
nx.draw(g)

In [ ]:
len(gp.d)

In [ ]:
len(gp.selection)

In [ ]:
gp.selection[10]['PDB'].iloc[0]

In [ ]:
len(gp.edges[0])

In [ ]:
gp.d[0]

In [ ]:
import networkx as nx
from torch_geometric.utils.convert import to_networkx

In [ ]:
g = to_networkx(gp.d[10], to_undirected=True)
nx.draw(g)

In [ ]:
len(gp.d)

In [ ]:
len(gp.selection)

In [ ]:
gp.selection[10]['PDB'].iloc[0]